In [47]:
import os
import cv2
import glob
import tqdm 
import numpy as np
import pandas as pd

import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset,DataLoader
from sklearn.model_selection import train_test_split

In [45]:
def load_image(image_address):
    im=cv2.imread(image_address)
    im=cv2.resize(im,(50,50))
    return im
def load_data(addresses):
    x=[]
    y=[]
    for address in tqdm.tqdm(addresses):
        x.append(load_image(address))
    return np.array(x)

In [52]:
breast_img_addresses = glob.glob('Dataset/BreastHistopathologyImages/*/*/*.png')
y=np.array(list(map(lambda x:float(x[-5]),breast_img_addresses)))
y=np.array(list(map(lambda x:x+0.00001 if x==0 else x-0.00001,y)))
X=load_data(breast_img_addresses)

100%|██████████| 277524/277524 [00:43<00:00, 6339.75it/s]


In [53]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [54]:
# ind=np.random.randint(X_train.shape[0])
# cv2.imshow(str(y_train[ind]),X_train[ind])
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [55]:
class P_dataset(Dataset):
    def __init__(self, X,y):
        self.X=X
        self.y=y
        
    def __getitem__(self, item):
            x=self.X[item]
            y=self.y[item]
            return torch.tensor(np.array(x)+1/255,dtype=torch.float32),torch.tensor(np.array(y),dtype=torch.float32)
    def __len__(self):
        return self.y.shape[0]

In [56]:
data_train=P_dataset(X_train,y_train)
data_test=P_dataset(X_test,y_test)
train_loader=DataLoader(data_train,batch_size=16,shuffle=True)
test_loader=DataLoader(data_test,batch_size=16,shuffle=False)

In [60]:
class ConvModel(nn.Module):
    def __init__(self):
        super(ConvModel, self).__init__()
        self.cnn1=nn.Conv2d(3,3,3)
        self.bn1=nn.BatchNorm2d(3)
        self.mp1=nn.MaxPool2d(2,stride=2)
        self.cnn2=nn.Conv2d(3,3,3)
        self.bn2=nn.BatchNorm2d(3)
        self.mp2=nn.MaxPool2d(2,stride=2)
        self.cnn3=nn.Conv2d(3,3,4)
        self.bn3=nn.BatchNorm2d(3)
        self.mp3=nn.MaxPool2d(2,stride=2)
        # self.cnn4=nn.Conv2d(12,24,3)
        # self.mp4=nn.MaxPool2d(2,stride=2)
        # self.cnn5=nn.Conv2d(24,48,3)
        # self.mp5=nn.MaxPool2d(2,stride=2)
        self.final=nn.Linear(3*4*4,1)
    def forward(self, x):
        x=torch.permute(x,(0,3,1,2))
        b=x.size(0)
        x=F.relu(self.bn1(self.cnn1(x)))
        x=self.mp1(x)
        x=F.relu(self.bn2(self.cnn2(x)))
        x=self.mp2(x)
        x=F.relu(self.bn3(self.cnn3(x)))
        x=self.mp3(x)
        # x=self.cnn4(x)
        # x=self.mp4(x)
        # x=self.cnn5(x)
        # x=self.mp5(x)
        x=x.reshape(b,-1)
        x=F.sigmoid(self.final(x))
        return x


In [61]:
BCELOSS=nn.BCELoss()
def loss_fn(y,y_hat):
    y_hat=y_hat.view(-1,)
    loss=BCELOSS(y,y_hat)
    acc=torch.sum((y>0.5)==(y_hat>0.5))/y.shape[0]
    return loss,acc

In [62]:
device='cpu'
model=ConvModel()

opt = torch.optim.Adam(model.parameters(),lr=0.01)

for epoch in range(100):
    train_loss=0
    train_acc=0
    test_loss=0
    test_acc=0
    train_len=0
    test_len=0
    model.train()
    for x,y in train_loader:
        train_len+=x.shape[0]
        x = x.to(device) # GPU
        # x=torch.permute(x,(0,3,1,2))
        y = y.to(device)
        # print(y.shape)
        opt.zero_grad()
        y_hat = model(x)
        # print(y_hat.shape)
        loss,acc=loss_fn(y,y_hat)
        train_loss+=loss.detach().numpy()
        train_acc+=acc
        loss.backward()

        opt.step()
    model.eval()    
    for x,y in test_loader:
        test_len+=x.shape[0]
        x = x.to(device) # GPU
        # x=torch.permute(x,(0,3,1,2))
        y = y.to(device)
        y_hat = model(x)
        loss,acc=loss_fn(y,y_hat)
        test_loss+=loss.detach().numpy()
        test_acc+=acc
    train_loss=train_loss/len(train_loader)
    train_acc=train_acc/len(train_loader)
    test_loss=test_loss/len(test_loader)
    test_acc=test_acc/len(test_loader)
    print(f"EPOCH {epoch}: train_loss: {train_loss}, test_loss: {test_loss}, train_acc: {train_acc}, test_acc: {test_acc},")
        

EPOCH 0: train_loss: 2.030174643465564, test_loss: 2.0141605643490585, train_acc: 0.8254531025886536, test_acc: 0.8253422379493713,
EPOCH 1: train_loss: 1.9371826329068564, test_loss: 2.3180466667312776, train_acc: 0.832021176815033, test_acc: 0.7989193201065063,
EPOCH 2: train_loss: 1.9148999127091042, test_loss: 2.1327455724575533, train_acc: 0.8338527679443359, test_acc: 0.8149495720863342,


KeyboardInterrupt: 